In [1]:
import numpy as np
import pandas as pd
import re, sys, os, csv
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from collections import Counter
from sklearn.base import BaseEstimator, TransformerMixin
from nltk.corpus import stopwords
from nltk.corpus import wordnet
from nltk.stem import PorterStemmer
from nltk.stem import WordNetLemmatizer
import nltk
import string    

In [6]:
def word_prob(word): return dictionary[word] / total
def words(text): return re.findall('[a-z]+', text.lower())
dictionary = Counter(words(open(r'C:\Users\Faizan Habib\Desktop\Emo2\Preprocessing\wordlist.txt').read()))
max_word_length = max(map(len, dictionary))
total = float(sum(dictionary.values()))

def viterbi_segment(text):
    probs, lasts = [1.0], [0]
    for i in range(1, len(text) + 1):
        prob_k, k = max((probs[j] * word_prob(text[j:i]), j)
                        for j in range(max(0, i - max_word_length), i))
        probs.append(prob_k)
        lasts.append(k)
    words = []
    i = len(text)
    while 0 < i:
        words.append(text[lasts[i]:i])
        i = lasts[i]
    words.reverse()
    return words, probs[-1]

def fix_hashtag(text):
    text = text.group().split(":")[0]
    text = text[1:] # remove '#'
    try:
        test = int(text[0])
        text = text[1:]
    except:
        pass
    output = ' '.join(viterbi_segment(text)[0])
    #print(output)
    return output


def clean_text(text):
    text = text.lower()
    text = re.sub(r"(#[A-Za-z0-9]+)", fix_hashtag, text)
    return ' '.join(re.sub(r"(@[A-Za-z0-9]+)|([^0-9A-Za-z \t])|(\w+:\/\/\S+)", " ", text).split())

def de_repeat(text):
    pattern = re.compile(r"(.)\1{2,}")
    return pattern.sub(r"\1\1", text)

def whitespaces(text):
    text = text.strip()
    return text

def remove_digits(text):
    return re.sub(r'\d+', '', text)

def remove_stopwords(text):
    stopwords_list = stopwords.words('english')
    # Some words which might indicate a certain sentiment are kept via a whitelist
    whitelist = ["n't", "not", "no"]
    words = text.split()
    clean_words = [word for word in words if (word not in stopwords_list or word in whitelist) and len(word) > 1]
    return " ".join(clean_words)

def stem_lemma(text):
    def get_wordnet_pos(word):
        tag = nltk.pos_tag([word])[0][1][0].upper()
        tag_dict = {"J": wordnet.ADJ, "N": wordnet.NOUN, "V": wordnet.VERB, "R": wordnet.ADV}
        return tag_dict.get(tag, wordnet.NOUN)     
        
    ps = PorterStemmer()
    words = text.split()
    stemmed_words = [ps.stem(word) for word in words]
    lemmatizer = WordNetLemmatizer()
    lemma_words = [lemmatizer.lemmatize(word, get_wordnet_pos(word)) for word in stemmed_words]
    return " ".join(lemma_words)

def vectorise_label(label):
    if label == "empty":return 1 # neutral
    elif label == "sadness":return 2 # sad
    elif label == "enthusiasm":return 3 # happy
    elif label == "neutral":return 0 # neutral
    elif label == "worry":return 4 # sad
    elif label == "surprise":return 5 # happy
    elif label == "love":return 6 # love
    elif label == "fun":return 7 # happy
    elif label == "hate":return 8 # anger
    elif label == "happiness":return 9 # happy
    elif label == "boredom":return 10 # neutral
    elif label == "relief":return 11 # happy
    elif label == "anger":return 12 #anger

In [3]:
data_train = pd.read_csv(r'C:\Users\Faizan Habib\Desktop\Emo2\Preprocessing\datasets\text_emotion.csv', sep=',')
print("Dataset shape:",data_train.shape)
print(data_train.sentiment[0],":",data_train.content[0])

Dataset shape: (40000, 4)
empty : @tiffanylue i know  i was listenin to bad habit earlier and i started freakin at his part =[


In [7]:
dataWriter = csv.writer(open(r'C:\Users\Faizan Habib\Desktop\Emo2\Preprocessing\datasets\cleaned_data.csv', 'w'), delimiter=',',lineterminator="\n")

total = 40000
for i in range(40000):
    print("Progress: ",round(i/total*100,2),"   ",end="\r")
    text = clean_text(data_train.content[i])
    text = de_repeat(text)
    text = whitespaces(text)
    text = remove_digits(text)
    text = remove_stopwords(text)
    text = stem_lemma(text)
    dataWriter.writerow([text, str(vectorise_label(data_train.sentiment[i]))])
    
print("Progress: ",100,"\nComplete!")

Progress:  100 0    
Complete!
